# QGraphs Data - Creation of digraphs

*This notebook creates* ***QGgraphs (Quantum Digraphs)*** *in* ***Networkx**.*

**By:** Jacob Cybulski<br>
**Date:** August 2024<br>
**Aims:** The goal of this notebook is to create random *Quantum Digraphs (QGraphs)*.<br/>
**Networkx technical refs:**
- Web site: https://networkx.org/documentation/stable/index.html
- Tutorial: https://networkx.org/documentation/stable/tutorial.html

In [ ]:
import sys
sys.path.append('.')
sys.path

In [ ]:
### General libraries

import matplotlib.pyplot as plt
import matplotlib.patches as pltpat
import pylab
import math
import os
from IPython.display import clear_output

from matplotlib import set_loglevel
set_loglevel("error")

### Get utilities
from utils.Digraphs import * 
from utils.Files import *

%matplotlib inline

## Define log details

In [ ]:
### Print options
#   0 - Print nothing
#   1 - Print essentials
#   2 - Print all
#   3 - Print debugging info

info_level = 1

In [ ]:
### Software version
MAJOR = 1
MINOR = 14

### Constants
LOG_NAME = 'logs'
DATA_NAME = 'gen'
DATA_PATH = f'{LOG_NAME}/data'
GRAPH_PATH = f'{LOG_NAME}/graph'
FIGURES_PATH = f'{LOG_NAME}/figures'

### Create a folder if needed
create_folder_if_needed(DATA_PATH)
create_folder_if_needed(GRAPH_PATH)
create_folder_if_needed(FIGURES_PATH)

### Show constants
if info_level > 0:
    print(f'LOG_NAME: {LOG_NAME}, DATA_NAME: {DATA_NAME}\n'+ 
          f'DATA_PATH: {DATA_PATH}, GRAPH_PATH: {GRAPH_PATH}, FIGURES_PATH: {FIGURES_PATH}')

## Networkx digraph manipulation

### Digraph generation

In [ ]:
from pylab import *
import json
import networkx as nx
from networkx.readwrite import json_graph

In [ ]:
### New graph parameters
vers = 11 # Change this value to add new graph, then set to zero for test runs
node_no = 16
edge_p = 0.1

In [ ]:
### Create a random graph
g = nx.gnp_random_graph(node_no, edge_p, directed=True)
g_node_no = g.number_of_nodes()
g_edge_no = g.number_of_edges()
unweighted_graph_name = f'digraph_{g_node_no:03d}_{g_edge_no:03d}_v{vers:03d}_unw'
print(f'Graph name: {unweighted_graph_name}')
draw_digraph(g)

### Save the graph

In [ ]:
### Saving of a graph to a file
g_path = f'{GRAPH_PATH}/{unweighted_graph_name}.json'
save_digraph(g, g_path)
print(f'Saved graph in file: {g_path}')

In [ ]:
digraph_details(g)

### Generate a QGraph for modeling

***Creation of a quantum digraph includes three stages:***<br/>
- *Converting an undirected graph to an adjacency matrix*
- *Changing all undirected edges to weighted directed edges*
- *Expanding a digraph to a QGraph (quantum digraph) by adding loops to stop vertices*

Note that on measurement, probabilities of navigation from a vertex to all possible targets adds to 1.

### Continue or load another graph

In [ ]:
### Select a graph and load it - either enter graph name or used the previous
# sel_graph_name = f'digraph_064_044_v010_unw' # change and comment the next line
sel_graph_name = f'digraph_{g_node_no:03d}_{g_edge_no:03d}_v{vers:03d}_unw'
sel_graph_name

In [ ]:
### Load the graph
g_path = f'{GRAPH_PATH}/{sel_graph_name}.json'
g_sel = load_digraph(g_path)

g_sel_node_no = g_sel.number_of_nodes()
g_sel_edge_no = g_sel.number_of_edges()
calc_p = g_sel_edge_no / g_sel_node_no**2
print(f'Digraph: name="{sel_graph_name}", node# = {g_sel_node_no}, edge# = {g_sel_edge_no} (calculated p = {np.round(calc_p, 2)})')

In [ ]:
### Define parameters of the graph to be loaded
save_vers = vers # Change this value to add new graph, then set to zero for test runs
save_graph_name = f'digraph_{g_sel_node_no:03d}_{g_sel_edge_no:03d}_v{save_vers:03d}_wei'
print(f'QGraph will be saved as: name="{save_graph_name}"')

In [ ]:
### Draw the selected and loaded graph
draw_digraph(g_sel, rcParams=(8, 6), 
             save_plot=f'{FIGURES_PATH}/{unweighted_graph_name}.eps')
print(f'Loaded graph from file: {g_path}')

### Generate a new QGraph

#### Testing the process step by step and observe results

In [ ]:
### Convert a graph to an adjacency matrix
import scipy as sci # Not used as yet
adj_sel = nx.adjacency_matrix(g_sel).toarray() # .todense() # adj_sel.toarray()
adj_sel

In [ ]:
### Expand the digraph to eliminate stop vertices (out-degree=0)
adj_exp = digraph_adj_expand(adj_sel)
np.around(adj_exp, 3)

In [ ]:
### Generate edge probability weights (adding to 1.0)
adj_w = digraph_adj_weigh(adj_exp, method='scale') # rand scale
np.around(adj_w, 3)

#### Now let us generate a QGraph in a single step

In [ ]:
### Test graph generation in one step
g_new = digraph_expanded_and_weighed(g_sel, method='rand') # scale or rand
draw_weighted_digraph(g_new, 'weight', ax=None, 
                      rcParams=(8, 6), save_plot=f'{FIGURES_PATH}/{save_graph_name}.eps')

In [ ]:
g_path = f'{GRAPH_PATH}/{save_graph_name}.json'
save_digraph(g_new, g_path)
print(f'Saved graph in file: {g_path}')

In [ ]:
digraph_details(g_new)

## System

In [ ]:
!pip list | grep -e torch -e PennyLane -e networkx